We will introduce how to analyze NYC Taxi dataset with Arctern, and use keplergl to display the data. 

First we need to load data:

In [1]:
import pandas as pd
nyc_schame={
    "VendorID":"string",
    "tpep_pickup_datetime":"string",
    "tpep_dropoff_datetime":"string",
    "passenger_count":"int64",
    "trip_distance":"double",
    "pickup_longitude":"double",
    "pickup_latitude":"double",
    "dropoff_longitude":"double",
    "dropoff_latitude":"double",
    "fare_amount":"double",
    "tip_amount":"double",
    "total_amount":"double",
    "buildingid_pickup":"int64",
    "buildingid_dropoff":"int64",
    "buildingtext_pickup":"string",
    "buildingtext_dropoff":"string",
}
nyc_df=pd.read_csv("/tmp/0_2M_nyc_taxi_and_building.csv",
               dtype=nyc_schame,
               date_parser=pd.to_datetime,
               parse_dates=["tpep_pickup_datetime","tpep_dropoff_datetime"])

display the pick-up point:

In [2]:
import arctern
from keplergl import KeplerGl

pickup_points = arctern.ST_Point(nyc_df.pickup_longitude,nyc_df.pickup_latitude)
KeplerGl(data={"pickup_points": pd.DataFrame(data={'pickup_points':arctern.ST_AsText(pickup_points)})})

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'pickup_points':                        pickup_points
0       POINT (-73.993003 40.747594)
1   …

The map results support select and drag, we can see it has noisy data, because the pick-up point are painted on the sea. In fact, all the data should be concentrated on land. These noisy data need to clean and filter.

In order to correctly analyze the NYC taxi data, we will filter the data according to the topographic map of New York City. The data that is not in the New York City map is regarded as noisy and needed filtered.

Read the topographic data map of New York City. The topographic data is stored in GeoJSON format. First, use Arctern to load the GeoJSON data:

In [3]:
import shapefile
import json
# read the topographic data map of New York City
nyc_shape = shapefile.Reader("/tmp/taxi_zones/taxi_zones.shp")
nyc_zone=[ shp.shape.__geo_interface__  for shp in nyc_shape.shapeRecords()]
nyc_zone=[json.dumps(shp) for shp in nyc_zone]
# read the data with Arctern
nyc_zone_series=pd.Series(nyc_zone)
nyc_zone_arctern=arctern.ST_GeomFromGeoJSON(nyc_zone_series)
arctern.ST_AsText(nyc_zone_arctern)

0    MULTIPOLYGON (((-8226155.13045259 4982269.9492...
1    MULTIPOLYGON (((-8243264.85067129 4948597.8364...
2    MULTIPOLYGON (((-8222843.67198779 4950893.7925...
3    MULTIPOLYGON (((-8219461.92460008 4952778.7319...
4    MULTIPOLYGON (((-8238858.86403699 4965915.0243...
dtype: object

Get the coordinate system of the current New York City topographic map, and use Arctern to convert the coordinate system to a latitude and longitude coordinate system, which is "EPSG: 4326":

In [4]:
from sridentify import Sridentify
ident = Sridentify()
ident.from_file('/tmp/taxi_zones/taxi_zones.prj')
src_crs = ident.get_epsg()
nyc_arctern_4326 = arctern.ST_Transform(nyc_zone_arctern,f'EPSG:{src_crs}','EPSG:4326')
arctern.ST_AsText(nyc_arctern_4326)

0    MULTIPOLYGON (((-73.8968088322377 40.795808445...
1    MULTIPOLYGON (((-74.0505080640325 40.566422034...
2    MULTIPOLYGON (((-73.8670614947212 40.582087976...
3    MULTIPOLYGON (((-73.8366827410671 40.594946697...
4    MULTIPOLYGON (((-74.0109284126803 40.684491472...
dtype: object

According to the converted latitude and longitude coordinates, the topographic map of New York City is drawn as follows:

In [5]:
KeplerGl(data={"nyc_zones": pd.DataFrame(data={'nyc_zones':arctern.ST_AsText(nyc_arctern_4326)})})

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'nyc_zones':                                            nyc_zones
0  MULTIPOLYGON (((-73.896808…

In order to analyze the taxi data in the New York City area, according to the topographic map of New York City, we see that the points that are not in the map are noise. In order to filter the noisy data, first we filter the pick-up points based on the skeleton map of New York City:

In [6]:
# this step will cost some time
nyc_arctern_one = arctern.ST_Union_Aggr(nyc_arctern_4326)
nyc_arctern_one = arctern.ST_SimplifyPreserveTopology(nyc_arctern_one,0.005)
is_in_nyc = arctern.ST_Within(pickup_points,nyc_arctern_one[0])
pickup_in_nyc = pickup_points[pd.Series(is_in_nyc)]

Display the pick-up point after data filtering:

In [7]:
KeplerGl(data={"pickup_points": pd.DataFrame(data={'pickup_points':arctern.ST_AsText(pickup_in_nyc)})})

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'pickup_points':                        pickup_points
0       POINT (-73.993003 40.747594)
1   …

According to the same method, filter the drop-off point:

In [8]:
# this step will cost some time
dropoff_points = arctern.ST_Point(nyc_df.dropoff_longitude,nyc_df.dropoff_latitude)
is_dorpoff_in_nyc = arctern.ST_Within(dropoff_points,nyc_arctern_one[0])
dropoff_in_nyc=dropoff_points[is_dorpoff_in_nyc]
KeplerGl(data={"drop_points": pd.DataFrame(data={'drop_points':arctern.ST_AsText(dropoff_in_nyc)})})

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'drop_points':                          drop_points
0       POINT (-73.983609 40.760426)
1     …

According to the latitude and longitude data of the pick-up point and the drop-off point, filter all noisy data on the dataset:

In [9]:
is_resonable = [is_dorpoff_in_nyc[idx] & is_in_nyc[idx] for idx in range(0,len(is_in_nyc)) ]
in_nyc_df=nyc_df[pd.Series(is_resonable)]
in_nyc_df.fare_amount.describe()

count    192690.000000
mean          9.733003
std           7.079953
min           2.500000
25%           5.700000
50%           7.700000
75%          11.000000
max         175.000000
Name: fare_amount, dtype: float64

We extract data with fees greater than $50 according to the transaction amount, and plot taxi pick-up and drop-off points:

In [10]:
fare_amount_gt_50 = in_nyc_df[in_nyc_df.fare_amount > 50]
pickup_50 = arctern.ST_Point(fare_amount_gt_50.pickup_longitude,fare_amount_gt_50.pickup_latitude)
dropoff_50 = arctern.ST_Point(fare_amount_gt_50.dropoff_longitude,fare_amount_gt_50.dropoff_latitude)
KeplerGl(data={"pickup": pd.DataFrame(data={'pickup':arctern.ST_AsText(pickup_50)}),
               "dropoff":pd.DataFrame(data={'dropoff':arctern.ST_AsText(dropoff_50)})
              })

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'pickup':                            pickup
0    POINT (-73.983795 40.737956)
1    POINT (-73.7…

We can also calculate the straight-line distance between the pick-up point and the drop-off point:

In [11]:
nyc_distance=arctern.ST_DistanceSphere(arctern.ST_Point(in_nyc_df.pickup_longitude,
                                                        in_nyc_df.pickup_latitude),
                                       arctern.ST_Point(in_nyc_df.dropoff_longitude,
                                                        in_nyc_df.dropoff_latitude))
nyc_distance.index=in_nyc_df.index
nyc_distance.describe()

count    192690.000000
mean       3134.342665
std        3287.665504
min           0.000000
25%        1224.352451
50%        2086.809469
75%        3744.095273
max       35395.487197
dtype: float64

Get the points with a straight-line distance greater than 20 kilometers, and draw all pick-up and drop-off points with a straight-line distance greater than 20 kilometers:

In [12]:
nyc_with_distance=pd.DataFrame({"pickup_longitude":in_nyc_df.pickup_longitude,
                                "pickup_latitude":in_nyc_df.pickup_latitude,
                                "dropoff_longitude":in_nyc_df.dropoff_longitude,
                                "dropoff_latitude":in_nyc_df.dropoff_latitude,
                                "sphere_distance":nyc_distance
                               })

nyc_dist_gt = nyc_with_distance[nyc_with_distance.sphere_distance > 20e3]
pickup_gt = arctern.ST_Point(nyc_dist_gt.pickup_longitude,nyc_dist_gt.pickup_latitude)
dropoff_gt = arctern.ST_Point(nyc_dist_gt.dropoff_longitude,nyc_dist_gt.dropoff_latitude)

KeplerGl(data={"pickup": pd.DataFrame(data={'pickup':arctern.ST_AsText(pickup_gt)}),
               "dropoff":pd.DataFrame(data={'dropoff':arctern.ST_AsText(dropoff_gt)})
              })

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'pickup':                             pickup
0     POINT (-73.781487 40.644855)
1     POINT (-7…